# Self Retrievers

![Self Retrievers](https://drive.google.com/uc?id=1OQUN-0MJcDUxmPXofgS7MqReEs720pqS)

Un recuperador autoconsultante puede analizar y entender las consultas que se le hacen en lenguaje natural, y luego, puede buscar y filtrar información relevante de su base de datos o documentos almacenados basándose en esas consultas. Esto lo hace transformando las consultas en un formato estructurado que puede interpretar y procesar de manera eficiente. Esto significa que, además de comparar la consulta del usuario con los documentos para encontrar coincidencias, también puede filtrar los resultados según criterios específicos extraídos de la consulta del usuario.

## Librerías

In [ ]:
from pprint import pprint

from dotenv import load_dotenv
from langchain.chains import create_tagging_chain_pydantic
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import SQLRecordManager, index
from langchain.retrievers import SelfQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pydantic import BaseModel, Field

from src.langchain_docs_loader import LangchainDocsLoader, num_tokens_from_string

load_dotenv()

## Carga de datos

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=400,
    chunk_overlap=50,
    length_function=num_tokens_from_string,
)

loader = LangchainDocsLoader(include_output_cells=False)
docs = loader.load()
docs = text_splitter.split_documents(docs)
len(docs)

In [ ]:
docs = [doc for doc in docs if doc.page_content != "```"]

## Inicializado de modelo de lenguaje

In [ ]:
llm = ChatOpenAI(temperature=0.1)

## Etiquetado de documentos

Los documentos por sí mismos son últiles, pero cuando son etiquetados con información adicional, pueden volverse más útiles. Por ejemplo, si etiquetamos los documentos con su idioma, podemos filtrar los documentos que no estén en el idioma que nos interesa. Si etiquetamos los documentos con su tema, podemos filtrar los documentos que no estén relacionados con el tema que nos interesa. De esta manera, podemos reducir el espacio de búsqueda y obtener mejores resultados.

### Creación de esquema de etiquetas

### Creación de cadena de generación de etiquetas (etiquetador)

In [ ]:
tagging_prompt = """Extract the desired information from the following passage.

Only extract the properties mentioned in the 'information_extraction' function.
Completness should involve more than one sentence.
To consider that a passage talks about a property, it is enough that it mentions it once.
If there is no mention of a property, set it to False. It only applies for the talk_about_* properties.

For instance,
To set `talks_about_vectorstore` to True, document should contain the word 'vectorstore' at least once.
To set `talks_about_retriever` to True, document should contain the word 'retriever' at least once.
To set `talks_about_chain` to True, document should contain the word 'chain' at least once.
To set `talks_about_expression_language` to True, document should contain the word 'expression language' or 'LCEL' at least once.

Passage:
{input}
"""

tagging_chain = create_tagging_chain_pydantic(Tags, llm)

### Ejemplos de uso del etiquetador

Probablemente, un fragmento que únicamente contiene una lista de enlaces a otros fragmentos que también se encuentran indexados no es muy útil. Esto podría ocasionar que recuperemos un documento que no es relevante para la consulta, mientra el documento que sí es relevante no se encuentraría en los primeros lugares de la lista de resultados.

In [ ]:
idx = 0

result = tagging_chain.invoke(input={"input": docs[idx].page_content})
print(result.get("input"))
pprint(result.get("text").dict())

Un fragmento con enlace a su documentación y ejemplo de uso sería más útil.

In [ ]:
idx = 1000

result = tagging_chain.invoke(input={"input": docs[idx].page_content})
print(result.get("input"))
pprint(result.get("text").dict())

In [ ]:
idx = 1400

result = tagging_chain.invoke(input={"input": docs[idx].page_content})
print(result.get("input"))
pprint(result.get("text").dict())

### Etiquetado de documentos

## Indexado de documentos

In [ ]:
vectorstore = Chroma(
    collection_name="langchain_docs",
    embedding_function=OpenAIEmbeddings(),
)

record_manager = SQLRecordManager(
    db_url="sqlite:///:memory:",
    namespace="chroma/langchain_docs",
)

record_manager.create_schema()

index(
    docs_source=docs_with_tags,
    record_manager=record_manager,
    vector_store=vectorstore,
    cleanup="full",
    source_id_key="source",
)

## Recuperación de documentos con un `Self Retriever`

### Creación de interfaz de los metadatos disponibles en el índice

### Creación de `retriever`

### Recuperación de documentos con el `retriever`

In [ ]:
relevant_documents = retriever.get_relevant_documents(
    "useful documents that talk about expression language and retrievers"
)
relevant_documents

In [ ]:
relevant_documents = retriever.get_relevant_documents(
    "useful documents that talk about expression language and retrievers or vectorstores"
)
relevant_documents